# RSA Integration Kit
This document demonstrates encryption and decryption of a file using a dedicated RSA accelerator for the PYNQ platform. It is developed as a part of the term project given in the course TFE4141 Design of Digital Systems 1 at the Department of Electronic Systems at the Norwegian University of Science and Technology.


# Digital design flow

Before starting the work on designing a digital circuit, a reminder of the digital hardware design flow is in place. 

1) Capture the requirements

2) Create a high level functional model. This model must produce the correct result, but is written at a very high abstraction level.

3) Design exploration. Use models at different abstraction levels. Try out different algorithms. What asymptotic complexity do they have? Is the problem parallelizable in any way? Can you use multiple cores and/or pipeline the design to increase the throughput?

4) Propose a microarchitecture at the register transfer level. The block diagram should be detailed enough that it identifies registers, muxes, adders and similar units. 

5) Estimate the performance and the area of your design. Estimate the clock frequency as well as the throughput.

6) Write the RTL kode for your design.

7) Verify functional correctness of the design. Use e.g. constrained random self checking testbenches or formal verification. 

8) Verify the performance of the design. Create special test in order to check that the performance is as expected.

9) Prototype the design e.g. on an FPGA. Do system testing.


# Constants
The following constants must be set to the appropriate values before attempting any encrypt/decrypt operations.

In [1]:
# ------------------------------------------------------------------------------
# Set the blocksize
# ------------------------------------------------------------------------------
C_BLOCKSIZE_IN_BITS         = 256
C_BLOCKSIZE_IN_32_BIT_WORDS = 8
C_BLOCKSIZE_IN_BYTES        = 32
WINDOW_SIZE                 = 4

# ------------------------------------------------------------------------------
# It is possible to choose between two different algorithms: 
#   XOR: This is the algorithm you can use for testing the system before you
#        have integrated the RSA hardware accelerator.
#        Computes: C=M xor n, M=C xor n
#
#   RSA: This is the algorithm you must enable when you have integrated the
#        RSA hardware accelerator.
#        Computes: C=M**e mod n, M=C**d mod n
# ------------------------------------------------------------------------------
C_ENCR_ALGORITHM_XOR = 0
C_ENCR_ALGORITHM_RSA = 1
C_ENCR_ALGORITHM = C_ENCR_ALGORITHM_XOR

# ------------------------------------------------------------------------------
# Import python libraries
# ------------------------------------------------------------------------------
import numpy as np
import time

# Utility functions

Messages will be read and written to files and sent in and out of the Pynq platform. In the following section a few utility functions for moving and converting data has been implemented.

In [2]:
# ------------------------------------------------------------------------------
# Function for converting an array of messages to a numpy array of 32-bit words
# ------------------------------------------------------------------------------
def msg2word(msg_array):
  word_array_array = []
  # Produce one word array per message
  for msg in msg_array: 
    msg_in_bytes = msg.to_bytes(C_BLOCKSIZE_IN_BYTES,byteorder='little')
    word_array_array.append(np.frombuffer(msg_in_bytes, dtype=np.uint32))
  
  # Concatentate the small word arrays into one large
  word_array = np.concatenate(word_array_array)    
  
  # Return the array of words
  return word_array

# ------------------------------------------------------------------------------
# Function for converting an numpy array of 32-bit words into messages
# ------------------------------------------------------------------------------
def word2msg(word_array):
  msg_array = []

  # Check that the message array size is a multiplum of 8 32 bit words
  word_array_length = len(word_array)
  assert word_array_length%C_BLOCKSIZE_IN_32_BIT_WORDS == 0, "The file size must be aligned to the block size" 
  message_count = int(word_array_length/C_BLOCKSIZE_IN_32_BIT_WORDS)

  # Loop over all messages
  for i in range(message_count):
    
    # Concatinate 32-bit words to a 256-bit message
    M = 0
    for j in range(C_BLOCKSIZE_IN_32_BIT_WORDS):
      M += (int(word_array[i*C_BLOCKSIZE_IN_32_BIT_WORDS+j]) << (j*32))
    
    # Append the message
    msg_array.append(M)
    
  # Return the message array
  return msg_array

# ------------------------------------------------------------------------------
# Function for testing msg2word word2msg conversion
# ------------------------------------------------------------------------------
def test_msg2msg():

  ma_in = [0x0000000011111111222222223333333344444444555555556666666677777777,
           0x8888888899999999aaaaaaaabbbbbbbbccccccccddddddddeeeeeeeeffffffff]
  wa = msg2word(ma_in)
  ma_out = word2msg(wa)
  if(ma_in==ma_out):
    print("test_msg2msg: PASSED")
  else:
    print("test_msg2msg: FAILED")
    
# ------------------------------------------------------------------------------
# Run a test of the utility functions
# ------------------------------------------------------------------------------
test_msg2msg()    

test_msg2msg: PASSED


# Message to encrypt/decrypt
The messages that will be encrypted/decrypted must all be downloaded from Blackboard and stored on the pynq platform. Copy the folder named "crypto" from Blackboard and paste it into the following location on the Pynq platform prior to running this jupyter notebook: 

/home/xilinx/pynq

The plaintex messages satisfy the following properties: 
* Property1: The size of the file must be a multiple of the block size, i.e. the filesize in bytes must be dividable by 32.
* Property2: C = M**e mod n, for 0 <= M < n. 

The second property has been satisfied through the following: 
* The files have been generated in such a way that they contain only ASCII characters with a byte value equal or lower than 127. The most significant bit in every byte is therefore 0, and thus it is also guaranteed that the MSB of any 256 bit blocks will be 0. 
* n is selected in such a way that the MSB of n is always 1.  

In [3]:
from pathlib import Path

# Two sets of testcases exists. One set used when the RSA algorithm is
# selected and another set for the XOR algorithm.
if(C_ENCR_ALGORITHM == C_ENCR_ALGORITHM_RSA):
  inp_msgdir = """/home/xilinx/pynq/crypto/rsa/inp_messages/"""
  otp_hw_msgdir = """/home/xilinx/pynq/crypto/rsa/otp_hw_messages/"""
  otp_sw_msgdir = """/home/xilinx/pynq/crypto/rsa/otp_sw_messages/"""    
else:
  inp_msgdir = """/home/xilinx/pynq/crypto/xor/inp_messages/"""
  otp_hw_msgdir = """/home/xilinx/pynq/crypto/xor/otp_hw_messages/"""
  otp_sw_msgdir = """/home/xilinx/pynq/crypto/xor/otp_sw_messages/"""    

# Name of the files to encrypt and decrypt
inp_files = ["pt0_in.txt", "pt1_in.txt", "pt2_in.txt", "ct3_in.txt", "ct4_in.txt", "ct5_in.txt"]
otp_files = ["ct0_out.txt", "ct1_out.txt", "ct2_out.txt", "pt3_out.txt", "pt4_out.txt", "pt5_out.txt"]
crypt_dir = ["ENCR", "ENCR", "ENCR", "DECR", "DECR", "DECR"]

# Testcase count
num_testcases = len(inp_files)

# Function for retrieving filenames for the different testcases
def get_testcase(testcase_sel):
  inp_file    = Path(inp_msgdir    + inp_files[testcase_sel])
  otp_hw_file = Path(otp_hw_msgdir + otp_files[testcase_sel])
  otp_sw_file = Path(otp_sw_msgdir + otp_files[testcase_sel])
  direction   = crypt_dir[testcase_sel]
  return direction, inp_file, otp_hw_file, otp_sw_file

# Check whether or not the files exist
for i in range(num_testcases):
  direction, inp_file, otp_hw_file, otp_sw_file = get_testcase(i)
  if not(inp_file.is_file()):
    print("File %s is missing. Download the file from Blackboard!" % str(inp_file))


File /home/xilinx/pynq/crypto/xor/inp_messages/pt0_in.txt is missing. Download the file from Blackboard!
File /home/xilinx/pynq/crypto/xor/inp_messages/pt1_in.txt is missing. Download the file from Blackboard!
File /home/xilinx/pynq/crypto/xor/inp_messages/pt2_in.txt is missing. Download the file from Blackboard!
File /home/xilinx/pynq/crypto/xor/inp_messages/ct3_in.txt is missing. Download the file from Blackboard!
File /home/xilinx/pynq/crypto/xor/inp_messages/ct4_in.txt is missing. Download the file from Blackboard!
File /home/xilinx/pynq/crypto/xor/inp_messages/ct5_in.txt is missing. Download the file from Blackboard!


# Keys and key generation

Pseudocode for RSA key generation is available here: https://repl.it/@Snesemann/RSAKeyGenAndEncrypt

When encrypting and decrypting the files in this script, the following keys will be used:

<pre>
n: 99925173 ad656867 15385ea8 00cd2812 0288fc70 a9bc98dd 4c90d676 f8ff768d
e: 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00010001
d: 0cea1651 ef44be1f 1f1476b7 539bed10 d73e3aac 782bd999 9a1e5a79 0932bfe9
</pre>

In [4]:
key_n = 0x99925173ad65686715385ea800cd28120288fc70a9bc98dd4c90d676f8ff768d
key_e = 0x0000000000000000000000000000000000000000000000000000000000010001
key_d = 0x0cea1651ef44be1f1f1476b7539bed10d73e3aac782bd9999a1e5a790932bfe9

# Software implementation of the RSA encryption algorithm
When designing digital circuits it is common to create a high level functional model. The following model is written at an extremely high level. It will produce correct results that can be used when generating test vectors for verification environments, but it does not provide any other value. It is expected that designers create more detailed models before attempting to write any RTL code. 


In [5]:
# ------------------------------------------------------------------------------
# Function for encrypting messages
# key_e  : 256 bit integer representing the key e
# key_n  : 256 bit integer representing the key n
# M_array: Array of 256-bit messages.
# ------------------------------------------------------------------------------
def sw_encrypt(key_e, key_n, M_array):
  
  # Initialize the array where we will store the ciphertext
  C_array = []

  # Start the timer
  start_time = time.time()

  # Loop over all messages
  for M in M_array:
    # Encrypt the message by computing C = M**key_e mod key_n
    # The result is a 256-bit ciphertext  
    if(C_ENCR_ALGORITHM == C_ENCR_ALGORITHM_RSA):    
      C = pow(M, key_e, key_n)
    
    # Encrypt the message by computing C = M xor key_n    
    else:
      C = M ^ key_n
        
    C_array.append(C)

  # Stop the timer
  stop_time = time.time()
  sw_exec_time = stop_time-start_time
        
  return C_array, sw_exec_time

# ------------------------------------------------------------------------------
# Function for decrypting messages
# key_d  : 256 bit integer representing the key e
# key_n  : 256 bit integer representing the key n
# C_array: Array of 256-bit messages.
# ------------------------------------------------------------------------------
def sw_decrypt(key_d, key_n, C_array):
  # Decryption is the same function as encryption, just with different keys
  return sw_encrypt(key_d, key_n, C_array)

# ------------------------------------------------------------------------------
# Function for testing encryption and decryption in software
# ------------------------------------------------------------------------------
def test_sw_encryptdecrypt(key_e, key_d, key_n):
  M_arr_in = [0x0000000011111111222222223333333344444444555555556666666677777777,
              0x8888888899999999aaaaaaaabbbbbbbbccccccccddddddddeeeeeeeeffffffff]
  C_arr, sw_encr_time = sw_encrypt(key_e, key_n, M_arr_in)
  M_arr_out, sw_decr_time = sw_decrypt(key_d, key_n, C_arr)
  
  if(M_arr_in == M_arr_out):
    print("test_hw_encryptdecrypt: PASSED, encr_time: %f, decr_time: %f" % (sw_encr_time, sw_decr_time))    
  else:
    print("test_sw_encryptdecrypt: FAILED")   
  
# ------------------------------------------------------------------------------
# Test that the software implementation of encryption and decryption works
# ------------------------------------------------------------------------------
test_sw_encryptdecrypt(key_e, key_d, key_n)
  

test_hw_encryptdecrypt: PASSED, encr_time: 0.000000, decr_time: 0.000000


# RSA encryption using an hardware accelerator

In the following code blocks, we test out the hardware RSA accelerator and measure it's performance.

In [6]:
# ------------------------------------------------------------------------------
# Loading drivers for the direct memory access block (DMA) that is responsible
# for reading the messages from memory and pushing them into the RSA design.
# ------------------------------------------------------------------------------
from pynq import Overlay
import pynq.lib.dma

# Load the overlay
overlay = Overlay('/home/xilinx/pynq/overlays/rsa_soc/rsa_soc.bit')

# Load the RSA DMA
dma = overlay.rsa.rsa_dma

# Load the MMIO driver for the RSA accelerator
rsammio = overlay.rsa.rsa_acc.mmio


<class 'ModuleNotFoundError'>: No module named 'pynq'

In [ ]:
# ------------------------------------------------------------------------------
# Function for writing C_BLOCKSIZE_IN_32_BIT_WORDS consequtive registers
# ------------------------------------------------------------------------------
def write_blockreg(address, block):
  reg_data = msg2word([block])
  addr = address
  # Each register contains 4 bytes. The registers are byte addressed.
  for data in reg_data:
    rsammio.write(addr, int(data))
    addr += 4

# ------------------------------------------------------------------------------
# Function for reading C_BLOCKSIZE_IN_32_BIT_WORDS consequtive registers
# ------------------------------------------------------------------------------
def read_blockreg(address):
  addr = address
  reg_arr = []
  for i in range(C_BLOCKSIZE_IN_32_BIT_WORDS):
    reg_arr.append(rsammio.read(addr))
    addr += 4
  reg_data = word2msg(reg_arr)[0]
  return reg_data
    
# ------------------------------------------------------------------------------
# Function for writing keys to the RSA accelerator
# ------------------------------------------------------------------------------

def write_keys(key_n=None, key_e_d=None,
               vlnw_schedule_0=None, vlnw_schedule_1=None, vlnw_schedule_2=None,
               R2_modn=None, n0_prime=None):
  """
  Writes any provided fields to the expected AXI-Lite addresses using your existing
  write_blockreg (which pairs with read_blockreg/word2msg/msg2word for consistent
  little-endian 32-bit word ordering).
  """
  # slv_reg(0..7)   : key_n            @ 0x00
  if key_n is not None:
    write_blockreg(0x00, key_n)
  # slv_reg(8..15)  : key_e_d          @ 0x20
  if key_e_d is not None:
    write_blockreg(0x20, key_e_d)
  # slv_reg(16..23) : r2_mod_n         @ 0x40
  if R2_modn is not None:
    write_blockreg(0x40, R2_modn)
  # slv_reg(24..31) : vlnw_schedule_0  @ 0x60
  if vlnw_schedule_0 is not None:
    write_blockreg(0x60, vlnw_schedule_0)
  # slv_reg(32..39) : vlnw_schedule_1  @ 0x80
  if vlnw_schedule_1 is not None:
    write_blockreg(0x80, vlnw_schedule_1)
  # slv_reg(40..47) : vlnw_schedule_2  @ 0xA0
  if vlnw_schedule_2 is not None:
    write_blockreg(0xA0, vlnw_schedule_2)
  # slv_reg(48)     : n_prime (32-bit) @ 0xC0
  if n0_prime is not None:
    rsammio.write(0xC0, int(n0_prime) & 0xFFFFFFFF)

# ------------------------------------------------------------------------------
# Function for reading keys from the RSA accelerator
# ------------------------------------------------------------------------------
def read_keys():
  # Read N, E
  n = read_blockreg(0x00)
  e_or_d = read_blockreg(0x20)
  return n, e_or_d

# ------------------------------------------------------------------------------
# Write the keys and read them back. Compare the result
# ------------------------------------------------------------------------------
def test_write_read_keys():
  write_keys(key_n=key_n, key_e_d=key_e)  # only write what the test reads back
  n, e = read_keys()
  if (key_n == n) and (key_e == e):
    print("test_write_read_keys: PASSED")
  else:
    print("test_write_read_keys: FAILED")

# ------------------------------------------------------------------------------
# Test writing keys to the accelerator and reading back the keys
# ------------------------------------------------------------------------------
test_write_read_keys()

In [ ]:
from pynq import Xlnk
import random
import numpy as np
"""
def vlnw_schedule(exponent: int, d: int = 4):

    e_bits = bin(exponent)[2:][::-1]  # LSB first
    schedule = []
    i = 0
    already_appended = False
    while i < len(e_bits):
        if e_bits[i] == '0':
            # Count consecutive zeros
            zero_count = 0
            while i < len(e_bits) and e_bits[i] == '0':
                zero_count += 1
                i += 1
                if zero_count == 4:
                    already_appended = True
                    schedule.append((0, zero_count))
                    zero_count = 0

            if already_appended != True:
                schedule.append((0, zero_count))
            already_appended = False
            # schedule.append((0, zero_count))

        else:
            win_len = min(d, len(e_bits) - i)
            val = 0
            for j in range(win_len):
                val |= (int(e_bits[i + j]) << j)
            schedule.append((val, win_len))
            i += win_len

    # Debug output
    print("Hardware-friendly schedule:")
    print(schedule)
    print("Total windows:", len(schedule))

    return schedule

def encode_vlnw_schedule(schedule):
    
    #Encode VLNW schedule into two 256-bit words.

    #Word 0 format:
      #[7 bits: schedule length] + [41 * 6-bit entries] + [3 zero bits]

    #Word 1 format:
      #[remaining entries] + [zero padding to 256 bits]
    

    # --- Sanity checks ---
    for val, ln in schedule:
        if val not in [0, 1, 3, 5, 7, 9, 11, 13, 15]:
            print(f"⚠️ Warning: Value {val} not in allowed set — may need adjustment.")
        if not (1 <= ln <= 4):
            raise ValueError(f"Length {ln} out of range (1–4)")

    # --- Encode as bitstring ---
    bitstream = format(len(schedule), "07b")
    for val, ln in schedule:
        val_bits = format(val, "04b")
        len_bits = format(ln - 1, "02b")
        bitstream += val_bits + len_bits

    total_bits = len(bitstream)
    print(f"\nTotal bits before splitting: {total_bits}")

    # --- Split into two 256-bit words ---
    first_payload_bits = 7 + 41 * 6  # 253 bits
    word0 = bitstream[:first_payload_bits]
    word1 = bitstream[first_payload_bits:]

    # Pad word0 to 256 bits with 3 zeros
    word0 = word0.ljust(256, '0')

    # Pad word1 to 256 bits (remaining entries or zeros)
    word1 = word1.ljust(256, '0')

    # --- Print breakdown ---
    print("\n--- ENCODED SCHEDULE BREAKDOWN ---")
    print(f"Entries total: {len(schedule)}")
    print(f"Word 0 bits ({len(word0)}): {word0}")
    print(f"Word 1 bits ({len(word1)}): {word1}")

    # Convert to integers
    w0_int = int(word0, 2)
    w1_int = int(word1, 2)

    print(f"\nWord0 (hex): 0x{w0_int:064x}")
    print(f"Word1 (hex): 0x{w1_int:064x}")

    return w0_int, w1_int

def modinv(a, m):
    #Modular inverse using extended Euclidean algorithm.
    g, x, y = extended_gcd(a, m)
    if g != 1:
        raise ValueError("No modular inverse")
    return x % m

def extended_gcd(a, b):
    #Extended GCD algorithm.
    if b == 0:
        return a, 1, 0
    g, y, x = extended_gcd(b, a % b)
    y -= (a // b) * x
    return g, x, y
"""

def schedule_lsb_sliding_packed(exp: int, w: int = 4) -> List[Tuple[int,int]]:
    """LSB-first (right-to-left) schedule of (u, L). Multiply by A^u then L squarings. Zeros packed up to w."""
    if exp <= 0:
        return []
    bits = [(exp >> i) & 1 for i in range(exp.bit_length())]  # LSB->MSB
    n, i, out = len(bits), 0, []
    while i < n:
        if bits[i] == 0:
            run = 0
            while i + run < n and bits[i + run] == 0:
                run += 1
            k = run
            while k > 0:
                L = min(w, k)
                out.append((0, L))
                k -= L
            i += run
        else:
            L = min(w, n - i)
            while L > 1 and bits[i + L - 1] == 0:
                L -= 1
            u = 0
            for j in range(L):
                u |= (bits[i + j] << j)  # LSB-first value
            if (u & 1) == 0 or u > 15:
                raise ValueError(f"Invalid window u={u} at bit {i}")
            out.append((u, L))
            i += L
    return out

def pack_three_regs(entries_msb: List[Tuple[int,int]]):
    """
    Pack MSB-first entries into three 256b regs for your controller:
      reg0[255:249] = length (7b, MSB first)
      reg0[248:3]   = first 246 payload bits (MSB-first)
      reg0[2:0]     = 000 (ignored)
      reg1[255:0]   = next 256 payload bits (MSB-first)
      reg2[255:0]   = next 256 payload bits (MSB-first)
    Returns (reg0_hex, reg1_hex, reg2_hex, length).
    """
    # encode each entry as 6 bits MSB-first: [u3 u2 u1 u0][ss1 ss0], where ss = L-1
    payload_bits = []
    for (u,L) in entries_msb:
        ss = (L - 1) & 0b11
        code = ((u & 0xF) << 2) | ss
        payload_bits.extend([(code >> k) & 1 for k in range(5, -1, -1)])  # MSB->LSB

    length = len(entries_msb)
    if length >= (1 << 7):
        raise ValueError("Entry count won't fit in 7 bits")

    # Build reg0
    reg0 = [ (length >> (6 - i)) & 1 for i in range(7) ]  # len MSB-first
    take0 = min(246, len(payload_bits))
    reg0 += payload_bits[:take0]
    reg0 += [0,0,0]  # ignored padding
    if len(reg0) < 256:
        reg0 += [0] * (256 - len(reg0))
    elif len(reg0) > 256:
        raise RuntimeError("reg0 overfilled (shouldn't happen)")

    # Build reg1
    rem = payload_bits[take0:]
    take1 = min(256, len(rem))
    reg1 = rem[:take1]
    if len(reg1) < 256:
        reg1 += [0] * (256 - len(reg1))
    rem = rem[take1:]

    # Build reg2
    take2 = min(256, len(rem))
    reg2 = rem[:take2]
    if len(reg2) < 256:
        reg2 += [0] * (256 - len(reg2))
    rem = rem[take2:]

    if rem:
        raise ValueError("Payload exceeds three 256-bit registers—add a 4th if needed")

    def bits_to_hex(msb_bits):
        v = 0
        for b in msb_bits:
            v = (v << 1) | b
        return f"0x{v:064x}"

    return bits_to_hex(reg0), bits_to_hex(reg1), bits_to_hex(reg2), length
    
def precompute_R2_modn__and_n0_prime(key_n):
  R = 1 << C_BLOCKSIZE_IN_BITS
  w = C_BLOCKSIZE_IN_BYTES
  n0 = key_n & ((1 << w) - 1)
  n0_inv = (-modinv(n0, 1 << w)) & ((1 << w) - 1)
  return R**2 % key_n, n0_inv

# ------------------------------------------------------------------------------
# Function that computes C = M**e mode n 
# key_e  : 256 bit integer representing the key e
# key_n  : 256 bit integer representing the key n
# M_array: Array 256-bit blocks.
# ------------------------------------------------------------------------------
def hw_encrypt(key_e, key_n, M_array):
  vlnw_schedule_0, vlnw_schedule_1, vlnw_schedule_2, length = pack_three_regs(schedule_lsb_sliding_packed(key_e, 4))
  R2_modn, n0_prime = precompute_R2_modn__and_n0_prime(key_n)
  # Write the keys
  write_keys(key_n, key_e, vlnw_schedule_0, vlnw_schedule_1, vlnw_schedule_2, R2_modn, n0_prime)
            
  # Allocate buffers for the input and output signals. 
  M_word_array = msg2word(M_array)
  xlnk = Xlnk()
  buffer_size_in_words = len(M_word_array)    
  print("Buffer size:", buffer_size_in_words)

  # The size of the files must be aligned to the block size of 256 bit = 8*32 bit words
  assert buffer_size_in_words%C_BLOCKSIZE_IN_32_BIT_WORDS == 0, "The file size must be aligned to the block size of 256 bit" 
    
  in_buffer  = xlnk.cma_array(shape=(buffer_size_in_words,), dtype=np.uint32)
  out_buffer = xlnk.cma_array(shape=(buffer_size_in_words,), dtype=np.uint32)

  # Copy the samples to the in_buffer
  np.copyto(in_buffer, M_word_array)

  # Trigger the DMA transfer and wait for the result
  # Waiting for completeness is done trough polling of registers. Measurements of
  # consumed time will be more accurate with the use of interrupts.
  start_time = time.time()
  dma.sendchannel.transfer(in_buffer)
  dma.recvchannel.transfer(out_buffer)
  #print("DMA.sendchannel complete")
  #dma.sendchannel.wait()
    
  #print("DMA.recvchannel complete") 
  # Should be sufficient to wait for the recieve channel to complete. This 
  # will reduce some of the polling overhead when running tests.
  dma.recvchannel.wait()
  stop_time = time.time()
  hw_exec_time = stop_time-start_time

  # Copy the result  
  C_word_array = out_buffer.copy()
  C_array = word2msg(C_word_array)
  
  # Free the buffers
  in_buffer.close()
  out_buffer.close()

  # Return the result
  return C_array, hw_exec_time

# ------------------------------------------------------------------------------
# Function that computes M = C**d mode n
#
# Encryption and decryption is the same operation. Thus the hw_decrypt function
# is redundant.
# ------------------------------------------------------------------------------
def hw_decrypt(key_d, key_n, C_array):
  return hw_encrypt(key_d, key_n, C_array)

# ------------------------------------------------------------------------------
# Function for testing encryption and decryption in hardware
# ------------------------------------------------------------------------------
def test_hw_encryptdecrypt(key_e, key_d, key_n):
  M_arr_in = [0x0000000011111111222222223333333344444444555555556666666677777777,
              0x8888888899999999aaaaaaaabbbbbbbbccccccccddddddddeeeeeeeeffffffff]
  C_arr, hw_encr_time = hw_encrypt(key_e, key_n, M_arr_in)
  M_arr_out, hw_decr_time = hw_decrypt(key_d, key_n, C_arr)
  
  if(M_arr_in == M_arr_out):
    print("test_hw_encryptdecrypt: PASSED, encr_time: %f, decr_time: %f" % (hw_encr_time, hw_decr_time))
  else:
    print("test_hw_encryptdecrypt: FAILED")
    print("M_arr_in: ", hex(M_arr_in[0]))   
    print("C_arr: ", hex(C_arr[0]))             
    print("M_arr_out: ", hex(M_arr_out[0]))                       
  
# ------------------------------------------------------------------------------
# Test that the hardware implementation of encryption and decryption works
# ------------------------------------------------------------------------------
test_hw_encryptdecrypt(key_e, key_d, key_n)

# Encrypt and decrypt long messages in hardware and software

Results from encrypting and decrypting files in hardware and software.

In [ ]:
# ------------------------------------------------------------------------------
# Loop over all files encrypt/decrypt and store the results.
# ------------------------------------------------------------------------------
hw_runtime = []
sw_runtime = []

for i in range(num_testcases):
  direction, inp_file, otp_hw_file, otp_sw_file = get_testcase(i)
  print("*"*80)
  print("CRYPT DIR       : ", direction)
  print("INPUT FILE      : ", inp_file)
  print("OUTPUT FILE (HW): ", otp_hw_file)
  print("OUTPUT FILE (SW): ", otp_sw_file)
  if(C_ENCR_ALGORITHM == C_ENCR_ALGORITHM_RSA):
    print("ENCR ALGORITHM  : RSA")
  else: 
    print("ENCR ALGORITHM  : XOR")    
  print("*"*80)
  input_msg_data                   = word2msg(np.fromfile(str(inp_file),dtype=np.uint32))
  if(direction == "ENCR"):
    output_msg_hw_data, hw_exec_time = hw_encrypt(key_e, key_n, input_msg_data)
    hw_runtime.append(hw_exec_time)
    output_msg_sw_data, sw_exec_time = sw_encrypt(key_e, key_n, input_msg_data)
    sw_runtime.append(sw_exec_time)
    print("HW RUNTIME: ", hw_exec_time)    
    print("SW RUNTIME: ", sw_exec_time)    
  elif(direction == "DECR"):
    output_msg_hw_data, hw_exec_time = hw_decrypt(key_d, key_n, input_msg_data)
    hw_runtime.append(hw_exec_time)
    output_msg_sw_data, sw_exec_time = sw_decrypt(key_d, key_n, input_msg_data)
    sw_runtime.append(sw_exec_time)
    print("HW RUNTIME: ", hw_exec_time)    
    print("SW RUNTIME: ", sw_exec_time)    
    
  # Compare results
  if(output_msg_sw_data == output_msg_hw_data):
    print("HW and SW produced the same result: TEST PASSED")
  else:
    print("HW and SW output did not match: TEST FAILED")
  print()

  # Write the results to file
  msg2word(output_msg_hw_data).tofile(str(otp_hw_file))
  msg2word(output_msg_sw_data).tofile(str(otp_sw_file))    
    
    

# Plot results


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

if(C_ENCR_ALGORITHM == C_ENCR_ALGORITHM_RSA):
  ALGORITHM = "RSA"
else: 
  ALGORITHM = "XOR"
    
runtime_plot = plt.figure()
ax = plt.subplot(111)
hw_testcases = [i-0.1 for i in range (len(inp_files))]
sw_testcases = [i+0.1 for i in range (len(inp_files))]
plt.ylabel('Time (sec)')
plt.xlabel('Testcase')
ax.bar(hw_testcases, hw_runtime,width=0.2,color='b',align='center', label='HW %s RUNTIME' % ALGORITHM)
ax.bar(sw_testcases, sw_runtime,width=0.2,color='r',align='center', label='SW %s RUNTIME' % ALGORITHM)
plt.legend()

# Discussion of results

## Discussion of XOR results
The overhead for running the HW encryption is relatively large. For small tests, such as 0,2,3 and 5, the performance of the HW solution is low compared to the SW solution. For the HW polling of registers is used for detecting when the encryption/decryption is completed. An improvement that will reduce the overhead is to implement support for interrupts.

Tests 1 and 4 are longer and here the HW is clearly faster than the SW solution since the overhead is no longer dominating the runtime of the HW.

## Discussion of RSA results
RSA is a much more demainding algorithm and should run slowly in software for most tests compared to the HW. 